In [183]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd

In [184]:
# Choose the appropriate web driver (e.g., Chrome, Firefox)
driver = webdriver.Chrome()  # You need to have the Chrome driver installed

# Navigate to the website
website_url = "https://calc.pcr24.ru/index.php"
driver.get(website_url)
df = pd.read_excel('/home/bio/projects/manhattan_pca/code/data/data_Ali.xlsx')
tmp = pd.read_csv('/home/bio/projects/manhattan_pca/code/data/R_stat_p.csv')
rss = tmp[tmp.codominant<=0.05].iloc[:,0].values

In [178]:

def get_res(values):
    try:
        def put(k,v):
            # Find and input value A
            key = f"input[name='{k}']"
            input_a = driver.find_element("css selector", key)
            input_a.clear()  # Clear any existing value
            input_a.send_keys(str(v))

        for k, v in values.items():
            put(k,v)

        calculate_button = driver.find_element("css selector", "input[name='submit']")
        calculate_button.click()

        driver.implicitly_wait(3)  # Wait for up to 10 seconds

        # Get the page source after calculations
        page_source = driver.page_source
        dfs = pd.read_html(page_source)
        df_ = []
        for i, name in zip([1,2,4,5], ['Allele', 'Codominant', 'Dominant', 'Recessive']):
            DF = dfs[i].set_index(dfs[i].columns[0])
            DF.columns = DF.iloc[0]
            DF = DF[1:]
            df_.append(name + '\n-----------------\n' +DF.to_string(col_space=10))

        return '\n\n'.join(df_)

    except Exception as e:
        print("An error occurred:", str(e))




In [185]:
def get_stats(d):
    a = d.index.str.cat()
    a,b = a[0], a[-1]


    values = {
        'a1':a, 
        'a2': b, 
        'sg1': d.iloc[0,0], 
        'sg2': d.iloc[1,0], 
        'sg3': d.iloc[2,0], 
        'cg1': d.iloc[0,1], 
        'cg2': d.iloc[1,1], 
        'cg3': d.iloc[2,1]}


    res = get_res(values)
    return res

results = ''
for rs in rss:
    
    res = df.pivot_table(index=rs, columns='Status', aggfunc='size', fill_value=0)
    res = rs + '\n' + get_stats(res) + '\n\n============================================================\n\n'
    results += res
    

In [186]:
# Close the browser
driver.quit()

In [188]:
with open("./data/confidence_calc24.txt", 'w') as f:
    f.write(results)

In [144]:
# print(res[0].to_markdown()) 

| 0        | Случаи   | Контроли   | χ2    | p      | OR    | OR          |
|:---------|:---------|:-----------|:------|:-------|:------|:------------|
| Аллели   | n = 126  | n = 40     | χ2    | p      | знач. | 95% CI      |
| Аллель A | 0.464    | 0.750      | 19.92 | 8.0E-6 | 0.29  | 0.16 – 0.51 |
| Аллель T | 0.536    | 0.250      | 19.92 | 8.0E-6 | 3.46  | 1.97 – 6.08 |


In [154]:
# print(res[0].to_string(col_space=10)) 

Аллели         Случаи   Контроли         χ2          p         OR           OR
0                                                                             
Аллели        n = 126     n = 40         χ2          p      знач.       95% CI
Аллель A        0.464      0.750      19.92     8.0E-6       0.29  0.16 – 0.51
Аллель T        0.536      0.250      19.92     8.0E-6       3.46  1.97 – 6.08


In [70]:
print(res[0].to_string(index=False, header=False))

  Аллели  Случаи Контроли    χ2      p    OR          OR
  Аллели n = 126   n = 40    χ2      p знач.      95% CI
Аллель A   0.464    0.750 19.92 8.0E-6  0.29 0.16 – 0.51
Аллель T   0.536    0.250 19.92 8.0E-6  3.46 1.97 – 6.08
